### First Login
After running this click Sign-In and login to Spotify. If the Time expires then repeat the process again.

In [1]:
from ipyauth import ParamsSpotify, Auth

p = ParamsSpotify(redirect_uri='http://localhost:8888/callback', client_id="9e4657eefbac41afa98c61f590d8fd51")
auth = Auth(params=p)
auth

Auth(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\x00\x00\x00\x02\xe…

In [38]:
access_token = auth.access_token
access_token

'BQBVdlZbmndxXHfNwb1jTih-Zuw7HekgivYHLUyqD55_gwINFKeYXJdvkwhA3ijXMJrdNlHXc_Ps8HM6b-18JGIR9WZrtnbzZh74pWPxtVRPh1q0LBmM-YY_yyWHP5y8q3EGAHby9mlG1pYfK8p6ShjBhVBvPtkDj0L-KOWhLQV--1uVZUYXXiXZt5bIWojZbgnp9NCa7lBrhUHfiAr4Rmu_9bwDwIGOTKMKI8dvoIQILCUwXOLBPlMAGG4'

### Common Stuff and Imports


In [49]:
!pip install pandas

     |████████████████████████████████| 10.4MB 144kB/s eta 0:00:01
     |████████████████████████████████| 20.0MB 5.5MB/s eta 0:00:01    |█████▊                          | 3.6MB 770kB/s eta 0:00:22     |██████████▉                     | 6.8MB 1.4MB/s eta 0:00:10
     |████████████████████████████████| 512kB 4.2MB/s eta 0:00:01


In [64]:
import requests
from IPython.display import Image
from pandas.io.json import json_normalize
from pandas import DataFrame,read_pickle
import logging
import os

logger = logging.getLogger()
logger.setLevel(logging.INFO)

def fetch( path, url=None ):
    callPath = url if (url!=None) else ("https://api.spotify.com" + path)
    response = requests.get(callPath , headers= {"Authorization":"Bearer " + access_token })
    if (response.status_code!=200):
        logging.debug("error ")
    return response.json()

def fetchPage( path, offset, limit):
    res = fetch(  path + "?offset=" + str(offset) + "&limit=" + str(limit))
    return res

def fetchAll( path ):
    more = fetchPage( path, 0, 50)
    limit = more["limit"]
    total = more["total"] - limit
    items = more["items"]
   # logging.info("length " + str(len(items)) + " to " + str(total))
    while((total>0) and (more["next"]!=None )):
        more = fetch( None, url = more["next"] )
        items.extend( more["items"])
        total = total - len(more["items"])
    return items

def fetchPageIds( path, ids): 
  res = fetch("{0}?ids={1}".format(path,",".join(ids)))
  return res

def fetchAllIds( path, resultField, ids, pageSize=50, existingDf=DataFrame.empty):
    if (not existingDf.empty):
        keys = existingDf.index.values
        ids = list(filter( lambda id: (id not in keys),ids))
    total = len(ids)
    logging.info("Requesting {0} rows".format(total))
    offset = 0
    artists = []
    while (offset < total) :
        result = fetchPageIds(path, ids[offset: min(total, offset + pageSize)])
        items = result[resultField]
        for item in items:
            existingDf.append(item)
        artists.extend(result[resultField]) 
        offset += pageSize
    return existingDf.reset_index().to_dict(orient="records")

user = fetch("/v1/me")
Image(url=user["images"][0]["url"], width=100)

### User Retreival
Get the user data and show the image

In [48]:
#data = fetchAll("/v1/me/tracks")
tracksDf = json_normalize(data, sep="_").set_index("track_id")
tracksDf.to_pickle("mytracks.pkl")
tracksDf.head()

,added_at,track_album_album_type,track_album_artists,track_album_available_markets,track_album_external_urls_spotify,track_album_href,track_album_id,track_album_images,track_album_name,track_album_release_date,...,track_popularity,track_preview_url,track_track_number,track_type,track_uri,track_linked_from_external_urls_spotify,track_linked_from_href,track_linked_from_id,track_linked_from_type,track_linked_from_uri
track_id,,,,,,,,,,,,,,,,,,,,,
1CS7Sd1u5tWkstBhpssyjP,2019-10-26T01:46:13Z,album,[{'external_urls': {'spotify': 'https://open.s...,"[AU, CA, US]",https://open.spotify.com/album/4Pv7m8D82A1Xun7...,https://api.spotify.com/v1/albums/4Pv7m8D82A1X...,4Pv7m8D82A1Xun7xNCKZjJ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Hozier (Expanded Edition),2014-09-19,...,78,https://p.scdn.co/mp3-preview/cdbb37ee18245800...,1,track,spotify:track:1CS7Sd1u5tWkstBhpssyjP,NaN,NaN,NaN,NaN,NaN
38Qqq67n2Z6yGANpKlCuKv,2019-10-25T17:19:41Z,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://open.spotify.com/album/2ESmheV4tLGuJil...,https://api.spotify.com/v1/albums/2ESmheV4tLGu...,2ESmheV4tLGuJilEJYk02Q,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Philip Glass: The Fall of the House of Usher,2019-10-18,...,25,https://p.scdn.co/mp3-preview/57b67f5ba41e2f61...,6,track,spotify:track:38Qqq67n2Z6yGANpKlCuKv,NaN,NaN,NaN,NaN,NaN
3RMuJ4qdkgs71mBNbPpZu0,2019-10-25T16:25:21Z,single,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://open.spotify.com/album/691WK2nkTQrNlq9...,https://api.spotify.com/v1/albums/691WK2nkTQrN...,691WK2nkTQrNlq9egbsJQw,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Swimmers,2019-10-25,...,48,https://p.scdn.co/mp3-preview/24e431807bfd082a...,1,track,spotify:track:3RMuJ4qdkgs71mBNbPpZu0,NaN,NaN,NaN,NaN,NaN
2WbrVeftLDcSAfQBHxA9P4,2019-10-25T16:15:45Z,single,[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",https://open.spotify.com/album/1sWcGlojlFQctnf...,https://api.spotify.com/v1/albums/1sWcGlojlFQc...,1sWcGlojlFQctnfRWlmUjI,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Blind Leading The Blind,2019-10-23,...,63,https://p.scdn.co/mp3-preview/e27317bf8a908a44...,1,track,spotify:track:2WbrVeftLDcSAfQBHxA9P4,NaN,NaN,NaN,NaN,NaN
48reQYlu36EpB24R9rIlxT,2019-10-25T16:08:23Z,single,[{'external_urls': {'spotify': 'https://open.s...,"[AE, AR, AU, BH, BO, BR, CA, CL, CO, CR, DO, E...",https://open.spotify.com/album/5HaKhstW9fj5wBB...,https://api.spotify.com/v1/albums/5HaKhstW9fj5...,5HaKhstW9fj5wBBkJEJx6t,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",One Slip (Remix 2019),2019-10-25,...,52,https://p.scdn.co/mp3-preview/a4624453758f6517...,1,track,spotify:track:48reQYlu36EpB24R9rIlxT,NaN,NaN,NaN,NaN,NaN


In [59]:
tracksDf = read_pickle("mytracks.pkl")
travk = tracksDf[["added_at","track_name","track_album_name"]]
#travk.to_dict(orient="records")
travk.head()

,added_at,track_name,track_album_name
track_id,,,
1CS7Sd1u5tWkstBhpssyjP,2019-10-26T01:46:13Z,Take Me to Church,Hozier (Expanded Edition)
38Qqq67n2Z6yGANpKlCuKv,2019-10-25T17:19:41Z,Act I Interlude,Philip Glass: The Fall of the House of Usher
3RMuJ4qdkgs71mBNbPpZu0,2019-10-25T16:25:21Z,Swimmers,Swimmers
2WbrVeftLDcSAfQBHxA9P4,2019-10-25T16:15:45Z,Blind Leading The Blind,Blind Leading The Blind
48reQYlu36EpB24R9rIlxT,2019-10-25T16:08:23Z,One Slip - Remix 2019,One Slip (Remix 2019)


### Create a Track to Artist Table
Pick out the track.artists array for each library track record. The meta (parent record) is the track.id. Use a prefix for both the meta and the record because they both use id.


In [29]:
#data[0]
artist_and_track = json_normalize( data=data, record_path=['track','artists'],  meta=[["track","id"]],  record_prefix='artist_',   sep="_" )
artist_and_track = artist_and_track[['track_id','artist_id']]
gbart = artist_and_track.groupby( by="track_id").count()
gbart.sort_values( by="track_id")
DataFrame.empty==DataFrame.empty

#len(artist_and_track["artist_id"].values)
#len(list(set(list(artist_and_track["sp_artist_id"].values))))

True

In [66]:
artistsPickle = read_pickle("artists.pkl") if (os.path.isfile("artists.pkl")) else DataFrame.empty 
artistIds = list(set(artist_and_track["artist_id"].values))
artists = fetchAllIds("/v1/artists","artists",artistIds,existingDf=artistsPickle)
artistsDf = json_normalize(artists).set_index("id")
artistsDf.to_pickle("artists.pkl")
artistsDf.head()

INFO:root:Requesting 0 rows


,genres,href,images,name,popularity,type,uri,external_urls.spotify,followers.href,followers.total
id,,,,,,,,,,
1vLFIIIwNjiO8VSNnh8nLh,[],https://api.spotify.com/v1/artists/1vLFIIIwNji...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Noordpool Orchestra,31,artist,spotify:artist:1vLFIIIwNjiO8VSNnh8nLh,https://open.spotify.com/artist/1vLFIIIwNjiO8V...,None,8258
46CitWgnWrvF9t70C2p1Me,"[chamber pop, electropop, indie folk, shimmer ...",https://api.spotify.com/v1/artists/46CitWgnWrv...,"[{'height': 1000, 'url': 'https://i.scdn.co/im...",Daughter,67,artist,spotify:artist:46CitWgnWrvF9t70C2p1Me,https://open.spotify.com/artist/46CitWgnWrvF9t...,None,964040
4aYc6yoP53lJmEeiRj9rNt,[],https://api.spotify.com/v1/artists/4aYc6yoP53l...,"[{'height': 1000, 'url': 'https://i.scdn.co/im...",John Butt,18,artist,spotify:artist:4aYc6yoP53lJmEeiRj9rNt,https://open.spotify.com/artist/4aYc6yoP53lJmE...,None,405
6zDFhpdu4WUVVGTz0JSLMh,"[acoustic pop, folk-pop]",https://api.spotify.com/v1/artists/6zDFhpdu4WU...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Susie Suh,49,artist,spotify:artist:6zDFhpdu4WUVVGTz0JSLMh,https://open.spotify.com/artist/6zDFhpdu4WUVVG...,None,34671
6s8AUvcmf2fp0Kh7PctPnd,[neo-classical],https://api.spotify.com/v1/artists/6s8AUvcmf2f...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",John Metcalfe,50,artist,spotify:artist:6s8AUvcmf2fp0Kh7PctPnd,https://open.spotify.com/artist/6s8AUvcmf2fp0K...,None,2124
